In [129]:
import pandas as pd
import numpy as np

#datadir = "../house-prices-advanced-regression-techniques/"
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

train = train.loc[train['GrLivArea'] < 4500] # outlier removal

for df in [train, test]:

    df.set_index("Id", inplace=True)
    df['KitchenQual'] = df['KitchenQual'].fillna('TA')
    df['MasVnrArea'] = df['MasVnrArea'].fillna(0)
    df['BsmtFullBath'] = df['BsmtFullBath'].fillna(0)
    df['BsmtHalfBath'] = df['BsmtHalfBath'].fillna(0)
    df['GarageArea'] = df['GarageArea'].fillna(0)
    df['TotalBsmtSF'] = df['TotalBsmtSF'].fillna(0)
    df['Total_porch_sf'] = (df['OpenPorchSF'] + df['3SsnPorch'] +df['EnclosedPorch'] + df['ScreenPorch'] + df['WoodDeckSF'])
    df['FullBath']  = df['FullBath'] + df['BsmtFullBath']
    df['HalfBath'] = df['HalfBath'] + df['BsmtHalfBath']
    
    df['TotSF'] = df['GrLivArea']-df['LowQualFinSF'] + df['TotalBsmtSF']

    df.drop(['FireplaceQu', 'Street', 'Utilities', 'LandContour', 
        'MasVnrType', 'Condition2', 'PoolArea', 'LotFrontage', 'CentralAir',
        'Functional', 'LandSlope', 'LotConfig', 'Fence', 'BldgType', 'Street',
        'Electrical', 'Alley', 'RoofStyle', 'KitchenAbvGr', 'BsmtFinType2',
        'Heating', 'PavedDrive', 'LandContour', 'Condition1', 'GarageCond', 
        'ExterCond', 'MSZoning', 'MiscFeature', 'SaleCondition', 'BsmtFinSF2',
        'SaleType', 'BsmtCond', 'MiscVal', 'GarageQual','EnclosedPorch',
        '3SsnPorch', 'RoofMatl', 'ScreenPorch', 'LowQualFinSF', 'Condition2',
        'PoolQC', 'PoolArea', 'BsmtHalfBath', 'BsmtFullBath', 'BsmtQual',
        'BsmtExposure','BsmtFinType1', 'BsmtFinSF1', 'BsmtUnfSF', '1stFlrSF', 
        '2ndFlrSF', 'MSSubClass', 'LotShape', 'Neighborhood', 'YearRemodAdd',
        'Exterior1st', 'Exterior2nd', 'GarageType', 'GarageYrBlt', 'Foundation',
        'GarageFinish', 'GarageCars','GrLivArea','TotalBsmtSF','OpenPorchSF',\
             'EnclosedPorch','3SsnPorch','WoodDeckSF','ScreenPorch',\
            'TotalBsmtSF'], axis=1, inplace=True) 

    

## performed a log transform of SalePrice
## https://www.kaggle.com/jesucristo/1-house-prices-solution-top-1
# train['LogSalePrice'] = np.log1p(train['SalePrice'])
train.to_csv('train_clean.csv', index=False)
test.to_csv('test_clean.csv', index=False)





In [130]:
test_df = pd.read_csv('test_clean.csv')

In [131]:
type(test_df.columns)

pandas.core.indexes.base.Index

In [138]:
test_df.shape

(1459, 27)

In [133]:
#Dummify
HS_dummy = pd.get_dummies(test_df['HouseStyle'], prefix='HS', prefix_sep='__', drop_first = True)
test_df = pd.concat([test_df.drop('HouseStyle', axis=1), HS_dummy], axis=1)
EQ_dummy = pd.get_dummies(test_df['ExterQual'], prefix='EQ', prefix_sep='__', drop_first = True)
test_df = pd.concat([test_df.drop('ExterQual', axis=1), EQ_dummy], axis=1)
HQC_dummy = pd.get_dummies(test_df['HeatingQC'], prefix='HQC', prefix_sep='__',drop_first = True)
test_df = pd.concat([test_df.drop('HeatingQC', axis=1), HQC_dummy], axis=1)
KQ_dummy = pd.get_dummies(test_df['KitchenQual'], prefix='KQ', prefix_sep='__',drop_first = True)
test_df = pd.concat([test_df.drop('KitchenQual', axis=1), KQ_dummy], axis=1)

In [134]:
test_df.columns

Index(['LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'MasVnrArea',
       'FullBath', 'HalfBath', 'BedroomAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageArea', 'MoSold', 'YrSold', 'Total_porch_sf', 'TotSF',
       'HS__1.5Unf', 'HS__1Story', 'HS__2.5Unf', 'HS__2Story', 'HS__SFoyer',
       'HS__SLvl', 'EQ__Fa', 'EQ__Gd', 'EQ__TA', 'HQC__Fa', 'HQC__Gd',
       'HQC__Po', 'HQC__TA', 'KQ__Fa', 'KQ__Gd', 'KQ__TA'],
      dtype='object')

In [135]:
test_df.drop(['HS__1.5Unf','HQC__Po','EQ__Gd','HS__1Story'],axis=1,inplace=True)

In [136]:
test_df.to_csv('test_clean.csv', index=False)

In [137]:
test_df.columns

Index(['LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'MasVnrArea',
       'FullBath', 'HalfBath', 'BedroomAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageArea', 'MoSold', 'YrSold', 'Total_porch_sf', 'TotSF',
       'HS__2.5Unf', 'HS__2Story', 'HS__SFoyer', 'HS__SLvl', 'EQ__Fa',
       'EQ__TA', 'HQC__Fa', 'HQC__Gd', 'HQC__TA', 'KQ__Fa', 'KQ__Gd',
       'KQ__TA'],
      dtype='object')